# 值迭代(Value Iteration)

In [1]:
import numpy as np
import pprint
import sys
from lib.envs.gridworld import GridworldEnv

## 載入遊戲

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

## 定義值迭代(Value Iteration)函數

In [3]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    env.P：狀態轉移機率矩陣
    env.P[s][a]：(prob, next_state, reward, done) 陣列
    env.nS：狀態總數
    env.nA：行動總數
    theta: 狀態值函數變動 <= theta，即停止進行。
    discount_factor: 獎勵折扣率(Gamma)
        
    Returns:
        傳回 (policy, V) 陣列. 
        - policy：是最佳策略, [S, A] 二為矩陣，含每個狀態/行動組合的機率分配.
        - V：狀態值函數。        
    """
    
    def one_step_lookahead(state, V):
        """
        state: 狀態值
        V: 狀態值函數
        
        傳回【行動值函數】
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A
    
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # 每一個狀態均採貪婪戰術
        for s in range(env.nS):
            # 走下一步的處理，計算行動值函數
            A = one_step_lookahead(s, V)
            # 找出最大行動值函數
            best_action_value = np.max(A)
            # 找出每一個位置的這一輪與上一輪狀態值函數差異最大者
            delta = max(delta, np.abs(best_action_value - V[s]))
            # 更新值函數
            V[s] = best_action_value        
        # 最大差異 < 事先設定值 theta，則停止評估
        if delta < theta:
            break
    
    # 再作一次，產生 policy
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # 走下一步的處理，計算行動值函數
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # 設定目前狀態下最佳的行動
        policy[s, best_action] = 1.0
    
    return policy, V

## 測試：顯示策略，即在每個狀態/行動組合的機率分配

In [ ]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

## 顯示狀態值函數

In [4]:
print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



## 顯示最後結果

In [5]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)